### Required Packages: 
Numpy, Numba and Pytorch

### Running this Notebook
Set the hyperparameters in cell [2]

### Running the experiments through command line

- for uncorrelated setting:
```
python train.py -n <num_agents> -l <lambda> -p <p_trunc>
```

- for correlated setting
```
python train_corr.py -n <num_agents> -l <lambda> -p <p_trunc> -c <p_corr>
```

In [1]:
import os
import sys
import time
import logging
import argparse
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F

from data import Data
from net import Net

### Set the hyperparamters here

In [2]:
# Hyperparameters
class HParams:
    def __init__(self):
        # Number of Agents
        self.num_agents = 4
        
        # Number of Hidden Layers and Hidden Nodes
        self.num_hidden_layers = 4
        self.num_hidden_nodes = 256
        
        # Learning Rate, Batch Size, Max iterations
        self.lr = 2e-3
        self.batch_size = 1024
        self.epochs = 50000
        
        # Printing Stats
        self.print_iter = 100
        self.val_iter = 1000
        
        # Save model @ iter
        self.save_iter = self.epochs - 1
        
        # Validation Batch Size
        self.num_val_batches = 200
        
        # Truncation probability
        self.prob = 0.2
        # Correlaiton probability
        self.corr = 0.25
        
        # Trade-off param
        self.lambd = 0.5
                
        # Seed
        self.seed = 0
        
        
# Initialize config
cfg = HParams()
device = "cuda:0"
np.random.seed(cfg.seed)

In [3]:
root_dir = os.path.join("experiments", "agents_%d"%(cfg.num_agents), "corr_%.2f"%(cfg.corr))
model_path = os.path.join(root_dir, "MODEL_%d_lambd_%f_prob_%.2f_corr_%.2f"%(cfg.seed, cfg.lambd, cfg.prob, cfg.corr))
os.makedirs(root_dir, exist_ok=True)
print("Model will be saved at %s"%(root_dir))

Model will be saved at experiments/agents_4/corr_0.25


In [4]:
# Data
G = Data(cfg)

# Neural Network
model = Net(cfg)
model.to(device)

Net(
  (input_block): Sequential(
    (0): Linear(in_features=32, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=256, out_features=256, bias=True)
    (9): LeakyReLU(negative_slope=0.01)
  )
  (layer_out_r): Linear(in_features=256, out_features=20, bias=True)
  (layer_out_c): Linear(in_features=256, out_features=20, bias=True)
)

In [5]:
def torch_var(x): return torch.Tensor(x).to(device)

# Loss functions
def compute_st(r, p, q):        
    wp = F.relu(p[:, :, None, :] - p[:, :, :, None])
    wq = F.relu(q[:, :, None, :] - q[:, None, :, :], 0)  
    t = (1 - torch.sum(r, dim = 1, keepdim = True))
    s = (1 - torch.sum(r, dim = 2, keepdim = True))
    rgt_1 = torch.einsum('bjc,bijc->bic', r, wq) + t * F.relu(q)
    rgt_2 = torch.einsum('bia,biac->bic', r, wp) + s * F.relu(p)
    regret =  rgt_1 * rgt_2 
    return regret.sum(-1).sum(-1).mean()/cfg.num_agents

def compute_ir(r, p, q):
    ir_1 = r * F.relu(-q)
    ir_2 = r * F.relu(-p)
    ir = ir_1 + ir_2
    return ir.sum(-1).sum(-1).mean()/cfg.num_agents

def compute_ic_FOSD(r, p, q, P, Q, r_mult = 1):
            
    IC_viol_P = torch.zeros(cfg.num_agents).to(device)
    IC_viol_Q = torch.zeros(cfg.num_agents).to(device)
    
    discount = torch_var((r_mult) ** np.arange(cfg.num_agents))

    for agent_idx in range(cfg.num_agents):

        P_mis, Q_mis = G.generate_all_misreports(P, Q, agent_idx = agent_idx, is_P = True, include_truncation = True)
        p_mis, q_mis = torch_var(P_mis), torch_var(Q_mis)
        r_mis = model(p_mis.view(-1, cfg.num_agents, cfg.num_agents), q_mis.view(-1, cfg.num_agents, cfg.num_agents))
        r_mis = r_mis.view(*P_mis.shape)

        r_diff = (r_mis[:, :, agent_idx, :] - r[:, None, agent_idx, :])*(p[:, None, agent_idx, :] > 0).float()
        _, idx = torch.sort(-p[:, agent_idx, :])
        idx = idx[:, None, :].repeat(1, r_mis.size(1), 1)
        
        fosd_viol = torch.cumsum(torch.gather(r_diff, -1, idx) * discount, -1)
        IC_viol_P[agent_idx] = F.relu(fosd_viol).max(-1)[0].max(-1)[0].mean(-1)

        P_mis, Q_mis = G.generate_all_misreports(P, Q, agent_idx = agent_idx, is_P = False, include_truncation = True)
        p_mis, q_mis = torch_var(P_mis), torch_var(Q_mis)
        r_mis = model(p_mis.view(-1, cfg.num_agents, cfg.num_agents), q_mis.view(-1, cfg.num_agents, cfg.num_agents))
        r_mis = r_mis.view(*Q_mis.shape)
        
        r_diff = (r_mis[:, :, :, agent_idx] - r[:, None, :, agent_idx])*(q[:, None, :, agent_idx] > 0).float()
        _, idx = torch.sort(-q[:, :, agent_idx])
        idx = idx[:, None, :].repeat(1, r_mis.size(1), 1)
        
        fosd_viol = torch.cumsum(torch.gather(r_diff, -1, idx) * discount, -1)
        IC_viol_Q[agent_idx] = F.relu(fosd_viol).max(-1)[0].max(-1)[0].mean(-1)

    IC_viol = (IC_viol_P.mean() + IC_viol_Q.mean())*0.5
    return IC_viol


In [6]:
# Optimizer
opt = torch.optim.AdamW(model.parameters(), lr = cfg.lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(opt, milestones=[10000,25000], gamma=0.5)

In [7]:
# Trainer
tic = time.time()
i = 0
while i < cfg.epochs:
  
    # Reset opt
    opt.zero_grad()
    model.train()
    
    # Inference
    if cfg.corr > 0:
        P, Q = G.generate_batch_with_corr(cfg.batch_size)
    else:
        P, Q = G.generate_batch(cfg.batch_size)
        
    p, q = torch_var(P), torch_var(Q)
    r = model(p, q)
    
    # Compute loss
    st_loss = compute_st(r, p, q)
    
    ic_loss = compute_ic_FOSD(r, p, q, P, Q)
        
    total_loss = st_loss * (cfg.lambd) + ic_loss * (1 - cfg.lambd)
    total_loss.backward() 
    
    opt.step()
    scheduler.step()
    t_elapsed = time.time() - tic

    
    # Validation  
    if i% cfg.print_iter == 0 or i == cfg.epochs - 1:
        print("[TRAIN-ITER]: %d, [Time-Elapsed]: %f, [Total-Loss]: %f"%(i, t_elapsed, total_loss.item()))
        print("[Stability-Viol]: %f, [IC-Viol]: %f"%(st_loss.item(), ic_loss.item()))
        
    if (i>0) and (i % cfg.save_iter == 0) or i == cfg.epochs - 1:
        torch.save(model.state_dict(), model_path)
        
    if ((i>0) and (i% cfg.val_iter == 0)) or i == cfg.epochs - 1:
        model.eval()
        with torch.no_grad():
            val_st_loss = 0.0
            val_ic_loss = 0.0
            for j in range(cfg.num_val_batches):
                P, Q = G.generate_batch_with_corr(cfg.batch_size)
                p, q = torch_var(P), torch_var(Q)
                r = model(p, q)
                st_loss = compute_st(r, p, q)
                ic_loss = compute_ic_FOSD(r, p, q, P, Q)
                val_st_loss += st_loss.item()
                val_ic_loss += ic_loss.item()
            print("\t[VAL-ITER]: %d, [ST-Loss]: %f, [IC-Loss]: %f"%(i, val_st_loss/cfg.num_val_batches, val_ic_loss/cfg.num_val_batches))
            
    i += 1

[TRAIN-ITER]: 0, [Time-Elapsed]: 0.534615, [Total-Loss]: 0.115795
[Stability-Viol]: 0.231356, [IC-Viol]: 0.000233
[TRAIN-ITER]: 100, [Time-Elapsed]: 28.971549, [Total-Loss]: 0.052593
[Stability-Viol]: 0.105065, [IC-Viol]: 0.000121
[TRAIN-ITER]: 200, [Time-Elapsed]: 57.402748, [Total-Loss]: 0.052567
[Stability-Viol]: 0.105017, [IC-Viol]: 0.000116


KeyboardInterrupt: 